In [0]:
%fs ls /databricks-datasets/COVID/covid-19-data

In [0]:
# 1) Look at what’s already on DBFS
dbutils.fs.ls('/databricks-datasets/COVID/covid-19-data/')

# 2) Define  Bronze path
bronze = "/tmp/bronze/covid_nyt"

# Drop & recreate
dbutils.fs.rm(bronze, recurse=True)
dbutils.fs.mkdirs(bronze)

# 3) Ingest all the CSVs into a single Bronze Delta
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

(
    spark.read
         .option("header", True)
         .option("inferSchema", True)
         .csv("dbfs:/databricks-datasets/COVID/covid-19-data/*.csv")
         .write
         .format("delta")
         .mode("overwrite")
         .save(bronze)
)

# 4) Verify it worked
print("Bronze row count:", spark.read.format("delta").load(bronze).count())
display(spark.read.format("delta").load(bronze).limit(5))

In [0]:
display( dbutils.fs.ls("/tmp/bronze/covid_nyt") )

In [0]:
%sql
--register bronze table into the Hive meta store 
USE CATALOG spark_catalog;    -- the legacy HMS catalog
USE SCHEMA default;           -- or whichever HM DB you prefer

CREATE TABLE IF NOT EXISTS bronze_covid_nyt
USING DELTA
LOCATION 'dbfs:/tmp/bronze/covid_nyt';


SHOW TABLES;

--%sql
--SHOW DATABASES;

--%sql
--SHOW TABLES IN exampleDB;

In [0]:
%python
#Create the look up table /.parquet
from pyspark.sql.functions import col
# 1. Read the full dataset (fact table)
df = spark.read.option("header", True).option("inferSchema", True).csv("dbfs:/databricks-datasets/COVID/covid-19-data/us-states.csv")

# 2. Extract distinct state→FIPS pairs
lookup_df = (df
  .select(col("state").alias("state"),
          col("fips").cast("string").alias("state_fips"))
  .where(col("fips").isNotNull())
  .distinct())

# 3. Write out just your lookup table
lookup_df.write.mode("overwrite").parquet("dbfs:/tmp/state_fips_lookup.parquet")

# Read it back
df2 = spark.read.parquet("dbfs:/tmp/state_fips_lookup.parquet")

# Console view
df2.show(truncate=False)

# Databricks table view
display(df2)

display(df2.orderBy("state_fips"))



In [0]:
%sql
--Create silver layer
-- Switch into your HMS catalog
USE CATALOG spark_catalog;
USE SCHEMA default;

-- 1) Clean & cast
CREATE OR REPLACE TABLE silver_covid_nyt_clean
USING DELTA
LOCATION 'dbfs:/tmp/silver/covid_nyt_clean'
AS
SELECT
  CAST(date   AS DATE)   AS event_date,
  state                  AS region,
  CAST(cases  AS INT)    AS cases,
  CAST(deaths AS INT)    AS deaths
FROM default.bronze_covid_nyt
WHERE date        IS NOT NULL
  AND state       RLIKE '^[A-Za-z ]+$' --state RLIKE '^[A-Za-z ]+$' drops every row where the region was purely numeric (e.g. “50”, “0”, “12382” …) or had any non-letter characters—so all those garbage region codes vanish.
  AND cases       IS NOT NULL
  AND deaths      IS NOT NULL;
--date IS NOT NULL, cases IS NOT NULL, deaths IS NOT NULL toss any records missing a date or a count.

-- 2) Deduplicate
CREATE OR REPLACE TABLE silver_covid_nyt_dedup
USING DELTA
LOCATION 'dbfs:/tmp/silver/covid_nyt_dedup'
AS
SELECT DISTINCT *
FROM silver_covid_nyt_clean;

-- 3) Enrich with FIPS (only once!)
CREATE OR REPLACE TABLE silver_covid_nyt
USING DELTA
LOCATION 'dbfs:/tmp/silver/covid_nyt'
AS
SELECT
  d.event_date,
  d.region,
  d.cases,
  d.deaths,
  l.state_fips
FROM silver_covid_nyt_dedup AS d
LEFT JOIN parquet.`dbfs:/tmp/state_fips_lookup.parquet` AS l
  ON d.region = l.state
WHERE l.state_fips IS NOT NULL;

-- Verify
SHOW TABLES IN default;
SELECT * FROM silver_covid_nyt LIMIT 5;


In [0]:
#Create Gold layer the key business metrics from your silver data. Typically you’ll want:
#State‐level cumulative summary (total cases & deaths to date)
#Daily metrics (new cases/deaths & 7-day rolling averages)

from pyspark.sql.functions import col, sum as _sum, max as _max, avg as _avg
from pyspark.sql.window import Window
# 1a. Read silver
silver = spark.read.format("delta").table("default.silver_covid_nyt")

# 1b. Build cumulative summary
# ---------------------------------------------------
# Gold Table 1: State‐level Cumulative Summary
# ---------------------------------------------------
cumulative = (
    silver
     .groupBy("region", "state_fips")
     .agg(
         _max("event_date").alias("last_report_date"),
         _sum("cases").alias("total_cases"),
         _sum("deaths").alias("total_deaths")
     )
)

# 1c. Write out as gold
# Write & register
cumulative.write \
    .format("delta") \
    .mode("overwrite") \
    .option("path", "dbfs:/tmp/gold/covid_state_summary") \
    .saveAsTable("default.gold_covid_state_summary")

# ---------------------------------------------------
# Gold Table 2: Daily Metrics with 7-Day Rolling Avg
# ---------------------------------------------------
# Define a 7-day window per region
w = Window.partitionBy("region").orderBy("event_date").rowsBetween(-6, 0)

daily_metrics = (
    silver
    .groupBy("event_date", "region", "state_fips")
    .agg(
        _sum("cases").alias("new_cases"),
        _sum("deaths").alias("new_deaths")
    )
    .withColumn("avg7_new_cases",  _avg("new_cases").over(w))
    .withColumn("avg7_new_deaths", _avg("new_deaths").over(w))
)

# Write & register
daily_metrics.write \
    .format("delta") \
    .mode("overwrite") \
    .option("path", "dbfs:/tmp/gold/covid_daily_metrics") \
    .saveAsTable("default.gold_covid_daily_metrics")

In [0]:
%sql
--Queries on Gold layer
-- List all rows (preview)
--gold_covid_state_summary should show one row per state with its FIPS code, the last report date, total cases and total deaths.
SELECT * 
FROM default.gold_covid_state_summary
LIMIT 10;

SELECT * 
FROM default.gold_covid_daily_metrics
LIMIT 10;

%sql 
SELECT * 
FROM default.silver_covid_nyt
LIMIT 10;

%sql
--Latest snapshot across all states
SELECT
  d.region,
  d.event_date,
  d.new_cases,
  d.avg7_new_cases,
  s.total_cases,
  s.total_deaths
FROM default.gold_covid_daily_metrics AS d
JOIN default.gold_covid_state_summary AS s
  ON d.region = s.region
WHERE d.event_date = (SELECT MAX(event_date) FROM default.gold_covid_daily_metrics)
ORDER BY d.new_cases DESC
LIMIT 10;

--Top 5 states by peak 7-day rolling average
%sql
SELECT
  region,
  MAX(avg7_new_cases) AS peak_avg7_cases
FROM default.gold_covid_daily_metrics
GROUP BY region
ORDER BY peak_avg7_cases DESC
LIMIT 5;

--Trend line for a specific state

%sql
SELECT
  event_date,
  new_cases,
  avg7_new_cases
FROM default.gold_covid_daily_metrics
WHERE region = 'New York'
ORDER BY event_date;


--Percentage of deaths on the latest date

%sql
WITH latest AS (
  SELECT * 
  FROM default.gold_covid_daily_metrics
  WHERE event_date = (SELECT MAX(event_date) FROM default.gold_covid_daily_metrics)
)
SELECT
  l.region,
  l.new_deaths,
  s.total_deaths,
  ROUND(100.0 * l.new_deaths / s.total_deaths, 2) AS pct_new_vs_total_deaths
FROM latest l
JOIN default.gold_covid_state_summary s
  ON l.region = s.region
ORDER BY pct_new_vs_total_deaths DESC
LIMIT 10;

